In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd

In [2]:
ids = pd.read_csv("../transients/transient_ids.csv", header=None)
ids = ids[0].values.tolist()

In [3]:
ids[:10]

['2021aetj',
 '2020aeio',
 '2020xu',
 '2021paz',
 '2018hyq',
 '2019gg',
 '2021ouk',
 '2019thz',
 '2020yef',
 '2021loa']

In [4]:
with open("../transients/data/transients_info.pickle", 'rb') as file:
    data = pickle.load(file)

In [5]:
data.head(5)

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
0,1,327.160083,-43.380019,17.800,1340.340,SN,SNIa,2018evo,ASAS-SN,1,1,1860.105,322.643
1,1,315.033417,-40.358594,17.869,1341.926,SN,SNIa,2018exc,ATLAS,1,1,528.652,1304.728
2,1,313.474583,-38.015594,18.781,1347.901,SN,SNII,2018fgb,ATLAS,1,1,424.192,1751.547
3,1,332.648000,-27.250161,16.700,1331.630,SN,SNII,2018emt,ASAS-SN,1,3,642.568,1607.839
4,1,327.584333,-26.300131,19.405,1337.965,AT,Unclassified,2018esp,ATLAS,1,3,1303.439,1157.924


In [6]:
data.shape

(4400, 13)

In [7]:
data["classification"].value_counts().reset_index()

,index,classification
0,Unclassified,3899
1,SNIa,333
2,SNII,69
3,SNIa-91T-like,17
4,CV,16
5,SNIIn,13
6,SNIc,7
7,SNIb,5
8,SNIc-BL,4
9,SNIa-91bg-like,4


In [8]:
data_ = data[data["IAU name"].isin(ids)]

In [9]:
data_.head()

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
0,1,327.160083,-43.380019,17.800,1340.340,SN,SNIa,2018evo,ASAS-SN,1,1,1860.105,322.643
1,1,315.033417,-40.358594,17.869,1341.926,SN,SNIa,2018exc,ATLAS,1,1,528.652,1304.728
2,1,313.474583,-38.015594,18.781,1347.901,SN,SNII,2018fgb,ATLAS,1,1,424.192,1751.547
3,1,332.648000,-27.250161,16.700,1331.630,SN,SNII,2018emt,ASAS-SN,1,3,642.568,1607.839
4,1,327.584333,-26.300131,19.405,1337.965,AT,Unclassified,2018esp,ATLAS,1,3,1303.439,1157.924


In [10]:
data_.shape

(3270, 13)

In [11]:
data__ = data_["classification"].value_counts().reset_index()

In [12]:
data__

,index,classification
0,Unclassified,2833
1,SNIa,287
2,SNII,64
3,CV,16
4,SNIa-91T-like,15
5,SNIIn,12
6,SNIc,7
7,SNIb,5
8,SNIc-BL,3
9,Other,3


In [13]:
column = ['Label','Counts']

In [14]:
data__.columns=column

In [15]:
data__.head()

,Label,Counts
0,Unclassified,2833
1,SNIa,287
2,SNII,64
3,CV,16
4,SNIa-91T-like,15


In [16]:
group ={
'SNIa': ['SNIa', 'SNIa-91bg-like', 'SNIa-91T-like','SNIa-pec','SNIa-SC'],
'Others': ['CV','Mdwarf','Other','Varstar','SLSN-I'],
'AGN':['AGN'],
'SNIb/c':['SNIb/c', 'SNIc-BL','SNIbn', 'SNIc','SNIb','SNI'],
'SNII': ['SNII', 'SNIIn', 'SNIIb', 'SNIIP', 'SNII-pec'],
'SN': ['SN'],
'Unclassified': ['Unclassified']
}

In [17]:
group

{'SNIa': ['SNIa', 'SNIa-91bg-like', 'SNIa-91T-like', 'SNIa-pec', 'SNIa-SC'],
 'Others': ['CV', 'Mdwarf', 'Other', 'Varstar', 'SLSN-I'],
 'AGN': ['AGN'],
 'SNIb/c': ['SNIb/c', 'SNIc-BL', 'SNIbn', 'SNIc', 'SNIb', 'SNI'],
 'SNII': ['SNII', 'SNIIn', 'SNIIb', 'SNIIP', 'SNII-pec'],
 'SN': ['SN'],
 'Unclassified': ['Unclassified']}

In [18]:
group__ = list()
for i in range(len(data__)):
    #print(data__.loc[i,"Label"])
    for k,v in group.items():
        if (data__.loc[i,"Label"] in v):
            group__.append(k)

In [19]:
len(group__)

24

In [20]:
group__

['Unclassified',
 'SNIa',
 'SNII',
 'Others',
 'SNIa',
 'SNII',
 'SNIb/c',
 'SNIb/c',
 'SNIb/c',
 'Others',
 'SNIa',
 'SNIb/c',
 'AGN',
 'Others',
 'Others',
 'SNIb/c',
 'SN',
 'SNII',
 'SNII',
 'SNIb/c',
 'SNII',
 'Others',
 'SNIa',
 'SNIa']

In [21]:
data__["Group"] = group__

In [22]:
data__

,Label,Counts,Group
0,Unclassified,2833,Unclassified
1,SNIa,287,SNIa
2,SNII,64,SNII
3,CV,16,Others
4,SNIa-91T-like,15,SNIa
5,SNIIn,12,SNII
6,SNIc,7,SNIb/c
7,SNIb,5,SNIb/c
8,SNIc-BL,3,SNIb/c
9,Other,3,Others


In [23]:
data__.to_csv(path_or_buf="../transients/labels__.csv", header=True, index=False)